# Import Event Log

In [1]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.conversion.log import converter as log_converter


if __name__ == "__main__":
    # Read the CSV file
    dataframe_log = pd.read_csv('../../data/logs/mobis.csv', sep=',')  

    # Drop the first column without knowing its name
    dataframe_log = dataframe_log.drop(dataframe_log.columns[0], axis=1)

    # Format the dataframe
    dataframe_log = pm4py.format_dataframe(
        dataframe_log, 
        case_id='case', 
        activity_key='activity', 
        timestamp_key='start'
    )

    # Convert the dataframe to event log
    log = log_converter.apply(dataframe_log)
    
dataframe_log

/var/folders/s_/ch_w_j2d0sqf6dbdc0_224m40000gq/T/ipykernel_11843/1672979557.py:15: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  dataframe_log = pm4py.format_dataframe(


,activity,start,end,type,user,travel_start,travel_end,case,cost,case:concept:name,concept:name,time:timestamp,@@index,@@case_index
0,file travel request,2017-01-17 11:17:00+00:00,2017-01-17 11:23:00+00:00,Employee,JB8510,2017-10-01 00:00:00+00:00,2017-01-15 00:00:00+00:00,105,NaN,105,file travel request,2017-01-17 11:17:00+00:00,0,0
1,check if travel request needs preliminary pric...,2017-01-17 11:23:00+00:00,2017-01-17 11:24:00+00:00,Employee,JB8510,2017-10-01 00:00:00+00:00,2017-01-15 00:00:00+00:00,105,NaN,105,check if travel request needs preliminary pric...,2017-01-17 11:23:00+00:00,1,0
2,decide on approval requirements,2017-01-17 11:24:00+00:00,2017-01-17 11:24:00+00:00,Employee,JB8510,2017-10-01 00:00:00+00:00,2017-01-15 00:00:00+00:00,105,NaN,105,decide on approval requirements,2017-01-17 11:24:00+00:00,2,0
3,check if booking is necessary,2017-01-17 11:24:00+00:00,2017-01-17 11:40:00+00:00,Travel Department,KS9688,2017-10-01 00:00:00+00:00,2017-01-15 00:00:00+00:00,105,NaN,105,check if booking is necessary,2017-01-17 11:24:00+00:00,3,0
4,check if expense documents exist,2017-01-18 05:59:00+00:00,2017-01-18 06:31:00+00:00,Employee,JB8510,2017-10-01 00:00:00+00:00,2017-01-15 00:00:00+00:00,105,NaN,105,check if expense documents exist,2017-01-18 05:59:00+00:00,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55804,confirm travel expense report,2017-11-22 06:48:00+00:00,2017-11-22 06:50:00+00:00,Employee,KI9211,2017-11-19 00:00:00+00:00,2017-11-20 00:00:00+00:00,6348,NaN,6348,confirm travel expense report,2017-11-22 06:48:00+00:00,55804,3353
55805,decide on travel expense approval,2017-11-22 12:59:00+00:00,2017-11-22 13:06:00+00:00,Manager,AK7488,2017-11-19 00:00:00+00:00,2017-11-20 00:00:00+00:00,6348,NaN,6348,decide on travel expense approval,2017-11-22 12:59:00+00:00,55805,3353
55806,send original documents to archive,2017-11-29 20:12:00+00:00,2017-11-29 20:24:00+00:00,Employee,KI9211,2017-11-19 00:00:00+00:00,2017-11-20 00:00:00+00:00,6348,NaN,6348,send original documents to archive,2017-11-29 20:12:00+00:00,55806,3353
55807,calculate payments,2017-12-08 09:32:00+00:00,2017-12-08 09:55:00+00:00,Accounting,FQ3758,2017-11-19 00:00:00+00:00,2017-11-20 00:00:00+00:00,6348,NaN,6348,calculate payments,2017-12-08 09:32:00+00:00,55807,3353


# Insert Start & End markers

In [2]:
# Define a function to insert start and end markers
def add_markers(df):
    # Identify unique case indices
    case_indices = df['@@case_index'].unique()
    
    # Prepare a container for new DataFrame rows
    new_rows = []
    
    # Iterate over each case index to add start and end markers
    for case_index in case_indices:
        # Create a start marker row with all columns except @@case_index set to 'Start'
        start_row = {col: 'Start' if col != '@@case_index' else case_index for col in df.columns}
        
        # Create an end marker row with all columns except @@case_index set to 'End'
        end_row = {col: 'End' if col != '@@case_index' else case_index for col in df.columns}
        
        # Append start row, rows for the current case, and end row
        new_rows.append(start_row)
        new_rows.extend(df[df['@@case_index'] == case_index].to_dict('records'))
        new_rows.append(end_row)
    
    # Convert the list of rows into a DataFrame
    return pd.DataFrame(new_rows)

# Apply the function to add start and end markers to the dataframe
modified_dataframe = add_markers(dataframe_log)

# Preprocess

In [3]:
codes, uniques = pd.factorize(modified_dataframe['activity'])
modified_dataframe['activity'] = codes

### Padding for Cases with less then 5 events

In [4]:
# Calculating the frequency of each unique value in '@@case_index'
frequency = modified_dataframe['@@case_index'].value_counts()

# Finding the minimum occurrence
min_occurrence = frequency.min()

min_occurrence

13

In [5]:
""" # Assuming your dataframe might have different types, let's create a generic function to add rows
def add_rows(group, num_rows, case_index_value):
    # For each column, determine the appropriate "zero" value (int 0, string '', etc.)
    additional_rows = pd.DataFrame({
        column: 0 if pd.api.types.is_numeric_dtype(group[column]) else '' for column in group.columns
    }, index=range(num_rows))
    
    # Set the @@case_index column to the current case index value
    additional_rows['@@case_index'] = case_index_value
    
    # Append the additional rows to the group
    return pd.concat([group, additional_rows], ignore_index=True)

# Function to pad cases with less than 5 events
def pad_cases(df):
    # Group by @@case_index
    groups = df.groupby('@@case_index')
    
    # Placeholder for modified groups
    modified_groups = []
    
    for name, group in groups:
        # Calculate the number of events to add
        events_to_add = 5 - len(group)
        
        if events_to_add > 0:
            # Add the required number of rows
            group = add_rows(group, events_to_add, name)
        
        # Append the modified group to the list
        modified_groups.append(group)
    
    # Concatenate all modified groups back into a single DataFrame
    return pd.concat(modified_groups, ignore_index=True)  """

' # Assuming your dataframe might have different types, let\'s create a generic function to add rows\ndef add_rows(group, num_rows, case_index_value):\n    # For each column, determine the appropriate "zero" value (int 0, string \'\', etc.)\n    additional_rows = pd.DataFrame({\n        column: 0 if pd.api.types.is_numeric_dtype(group[column]) else \'\' for column in group.columns\n    }, index=range(num_rows))\n    \n    # Set the @@case_index column to the current case index value\n    additional_rows[\'@@case_index\'] = case_index_value\n    \n    # Append the additional rows to the group\n    return pd.concat([group, additional_rows], ignore_index=True)\n\n# Function to pad cases with less than 5 events\ndef pad_cases(df):\n    # Group by @@case_index\n    groups = df.groupby(\'@@case_index\')\n    \n    # Placeholder for modified groups\n    modified_groups = []\n    \n    for name, group in groups:\n        # Calculate the number of events to add\n        events_to_add = 5 - len(

In [6]:
""" # Apply the padding function
modified_dataframe = pad_cases(modified_dataframe)
modified_dataframe """

' # Apply the padding function\nmodified_dataframe = pad_cases(modified_dataframe)\nmodified_dataframe '

# Generate sliding windows

In [7]:
df_activity = modified_dataframe[['activity', '@@case_index']]

In [8]:
def generate_sliding_windows(df, case_id_column='@@case_index', window_size=5):
    windows = []
    targets = []
    case_indices = []

    # Iterate over each unique case
    for case_id in df[case_id_column].unique():
        # Extract the case
        case_data = df[df[case_id_column] == case_id]
        
        # Convert case_data to a NumPy array and drop the case_id_column
        case_data_array = case_data.drop(columns=[case_id_column]).to_numpy()

        # Adjusting the condition to correctly reflect window_size without needing an additional +1
        # Now it correctly considers window_size as including the target event
        if len(case_data_array) >= window_size:
            # Adjust the loop to generate sliding windows of size window_size - 1 for the inputs and use the next event as the target
            for i in range(len(case_data_array) - window_size + 1):
                # window now has window_size - 1 events
                window = case_data_array[i:i + window_size - 1]
                # The target is the event immediately following the window
                target = case_data_array[i + window_size - 1]
                windows.append(window)
                targets.append(target)
                case_indices.append(case_id)  # Store the case_id corresponding to the window

    # Convert lists to numpy arrays for easier handling and to ensure they are two-dimensional
    windows_array = np.array(windows)
    targets_array = np.array(targets)
    case_indices_array = np.array(case_indices)
    
    return windows_array, targets_array, case_indices_array

In [9]:
windows_activity, targets_activity, case_indices = generate_sliding_windows(df_activity)

# LSTM

### Architecture

In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Assuming these values as placeholders, replace them with actual counts from your data
num_activities = modified_dataframe['concept:name'].nunique()

embedding_dim_activity = 50
time_steps = 4

# Input layer for activities
input_activity = Input(shape=(time_steps,), name='input_activity')

# Embedding layer for activities
embedding_activity = Embedding(input_dim=num_activities, output_dim=embedding_dim_activity, input_length=time_steps)(input_activity)

# LSTM layer for activities
lstm_activity = LSTM(25, return_sequences=False)(embedding_activity)

# Output layer for predicting the next activity
output_activity = Dense(num_activities, activation='softmax', name='output_activity')(lstm_activity)

# Create and compile the model
model = Model(inputs=input_activity, outputs=output_activity)

model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

2024-08-19 21:53:58.143129: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_activity (InputLayer  [(None, 4)]               0         
 )                                                               
                                                                 
 embedding (Embedding)       (None, 4, 50)             1400      
                                                                 
 lstm (LSTM)                 (None, 25)                7600      
                                                                 
 output_activity (Dense)     (None, 28)                728       
                                                                 
Total params: 9728 (38.00 KB)
Trainable params: 9728 (38.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Data Splitting

In [11]:
from sklearn.model_selection import train_test_split

# Split the activity data
train_activity, test_activity, train_targets_activity, test_targets_activity = train_test_split(
    windows_activity, targets_activity, test_size=0.3, random_state=42)

# Training

In [12]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler

# EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

def cyclic_lr(epoch, lr):
    # Example function that modulates LR within a range for each epoch
    # Customize this function based on your cyclic learning rate policy
    max_lr = 0.01  # Maximum LR
    base_lr = 0.001  # Base LR
    step_size = 10  # Number of epochs for half a cycle
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = base_lr + (max_lr - base_lr) * np.maximum(0, (1 - x))
    return lr

lr_scheduler = LearningRateScheduler(cyclic_lr)

In [13]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# Convert activity targets to categorical
train_targets_activity_cat = to_categorical(train_targets_activity, num_classes=num_activities)
test_targets_activity_cat = to_categorical(test_targets_activity, num_classes=num_activities)

# Train the model
history = model.fit(train_activity, 
                    train_targets_activity_cat,
                    epochs=25,
                    batch_size=64,
                    validation_data=(test_activity, test_targets_activity_cat),
                    verbose=1,
                    callbacks=[lr_scheduler])  # Add other callbacks as needed

Epoch 1/25
538/538 [==============================] - 4s 4ms/step - loss: 1.2329 - accuracy: 0.6959 - val_loss: 0.3810 - val_accuracy: 0.8855 - lr: 0.0010
Epoch 2/25
538/538 [==============================] - 2s 3ms/step - loss: 0.3034 - accuracy: 0.8802 - val_loss: 0.2647 - val_accuracy: 0.8917 - lr: 0.0019
Epoch 3/25
538/538 [==============================] - 2s 3ms/step - loss: 0.2647 - accuracy: 0.8851 - val_loss: 0.2524 - val_accuracy: 0.8926 - lr: 0.0028
Epoch 4/25
538/538 [==============================] - 2s 3ms/step - loss: 0.2570 - accuracy: 0.8848 - val_loss: 0.2499 - val_accuracy: 0.8885 - lr: 0.0037
Epoch 5/25
538/538 [==============================] - 2s 3ms/step - loss: 0.2520 - accuracy: 0.8855 - val_loss: 0.2422 - val_accuracy: 0.8931 - lr: 0.0046
Epoch 6/25
538/538 [==============================] - 2s 3ms/step - loss: 0.2509 - accuracy: 0.8859 - val_loss: 0.2466 - val_accuracy: 0.8935 - lr: 0.0055
Epoch 7/25
538/538 [==============================] - 2s 3ms/step - lo

In [14]:
# Evaluate the model using only the activity-related data
evaluation = model.evaluate(
    test_activity,
    test_targets_activity_cat
)

print(evaluation)

461/461 [==============================] - 1s 1ms/step - loss: 0.2379 - accuracy: 0.8938
[0.2379106730222702, 0.893829345703125]


# Anomaly Score Computation

In [15]:
# Generate predictions for all inputs
predictions_activity = model.predict([windows_activity])

1535/1535 [==============================] - 2s 1ms/step


In [16]:
# Step 2: Compute Anomaly Scores for categorical variables (Resources and Activities)

def compute_anomaly_scores(predictions, actuals):
    # For categorical predictions, convert actuals to one-hot for comparison
    actuals_one_hot = to_categorical(actuals, num_classes=predictions.shape[-1])
    
    max_predictions = np.max(predictions, axis=-1)
    actual_predictions = np.sum(predictions * actuals_one_hot, axis=-1)  # Extract the probability of the actual class
    
    anomaly_scores = (max_predictions - actual_predictions) / max_predictions
    
    return anomaly_scores

anomaly_scores_activity = compute_anomaly_scores(predictions_activity, targets_activity)

In [17]:
import numpy as np

def classify_cases(anomaly_scores_activity, threshold=0.98):
    # Ensure all inputs are numpy arrays of the same shape
    anomaly_scores_activity = np.array(anomaly_scores_activity).flatten()

    # Find the maximum anomaly score across all attributes for each case
    max_scores = np.maximum.reduce([anomaly_scores_activity])

    # Classify cases as anomalous if the maximum anomaly score exceeds the threshold
    anomalous_cases = max_scores > threshold
    
    return anomalous_cases

# Now use the anomaly scores for user, activity, type, and timestamp in the classification
anomalous_cases = classify_cases(anomaly_scores_activity)

# Mapping

### True: anomaly, False: no anomaly

In [18]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
mapping = pd.DataFrame({'case': case_indices})
mapping['predicted'] = anomalous_cases
mapping

,case,predicted
0,0,False
1,0,False
2,0,False
3,0,False
4,0,False
...,...,...
49096,3353,False
49097,3353,False
49098,3353,False
49099,3353,False


In [19]:
case_prediction = mapping.groupby('case')['predicted'].any()
case_prediction

case
0       False
1       False
2       False
3       False
4       False
        ...  
3349    False
3350    False
3351    False
3352    False
3353    False
Name: predicted, Length: 3354, dtype: bool

# Ground Truth

In [20]:
def generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking):
    from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
    from pm4py.algo.conformance.alignments.petri_net import variants
    from pm4py.objects.petri_net.utils import align_utils
    max_events=0
    for trace in log:
        counter=0
        for event in trace:
            counter+=1
        if counter > max_events:
            max_events=counter
    parameters={}
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_SYNC_COST_FUNCTION] = list(map(lambda i: .1*i, range(max_events*2)))
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_TRACE_COST_FUNCTION]=list(map(lambda i: align_utils.STD_MODEL_LOG_MOVE_COST-.1*i, range(max_events*2)))
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking, variant=variants.state_equation_a_star, parameters=parameters)
    return aligned_traces

In [21]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/model/MobisToBe.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_traces = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/295 [00:00<?, ?it/s]

In [22]:
def extract_conformance_status_by_fitness(aligned_traces):
    conformance_status = []
    for alignment in aligned_traces:
        fitness = alignment['fitness']
        # If the fitness is 1.0, the trace is conforming
        if fitness == 1.0:
            conformance_status.append(0)
        else:
            conformance_status.append(1)
    return conformance_status

# Get the conformance status list from the aligned traces
conformance = extract_conformance_status_by_fitness(aligned_traces)

In [23]:
ground_truth = pd.DataFrame({'conformity': conformance})
ground_truth['predicted'] = case_prediction

# Convert False to 0 and True to 1
ground_truth['predicted'] = [int(value) for value in ground_truth['predicted']]
ground_truth

,conformity,predicted
0,0,0
1,1,0
2,0,0
3,1,0
4,1,0
...,...,...
3349,0,0
3350,1,0
3351,0,0
3352,0,0


# Evaluation

In [24]:
# Calculating TP, TN, FP, FN
TP = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 1)).sum()
TN = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 0)).sum()
FP = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 1)).sum()
FN = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 0)).sum()

In [25]:
precision_dev = TP / (TP + FP)
print(f"Precision Dev: {precision_dev:.2f}")

Precision Dev: 1.00


In [26]:
recall_dev = TP / (TP + FN)
print(f"Recall Dev: {recall_dev:.2f}")

Recall Dev: 0.07


In [27]:
precision_no_dev = TN / (TN + FN)
print(f"Precision No Dev: {precision_no_dev:.2f}")

Precision No Dev: 0.52


In [28]:
recall_no_dev = TN / (TN + FP)
print(f"Recall No Dev: {recall_no_dev:.2f}")

Recall No Dev: 1.00


In [29]:
from sklearn.metrics import roc_auc_score

auc_roc = roc_auc_score(ground_truth['conformity'], ground_truth['predicted'])
print(f"AUC-ROC: {auc_roc:.2f}")

AUC-ROC: 0.53
